In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("Lab_5_matrices.ipynb")

# Using matrices to position parts of a robot arm

The goal of this part of the assignment is to use matrices to position a robot arm in space. In the lab we'll just
position each component independently and rotate them by angles.


Slides: https://docs.google.com/presentation/d/1Ut5RnIKU8DF8k_joGXp4tJ1FzBKNIX8JYRE9wkIP_qE/edit?

In [ ]:
# The usual imports
import numpy as np
import matplotlib.pyplot as plt

# These are the routines used in the lecture activity - we'll re-use them here
import matrix_routines as mt
from object_routines import read_object, plot_object_in_world_coord_system

In [ ]:
# These are the routines you'll edit. You can either copy them in from the .py file OR copy here, edit, then put it 
# back in the .py file
from arm_routines import get_rotation_link, get_matrix_finger, get_transform_base, get_transform_link, get_transform_palm, \
    get_transform_finger, create_arm_geometry, set_angles_of_arm_geometry, plot_arm_components


In [ ]:
# These commands will force JN to actually re-load the external file when you re-execute the import command
%load_ext autoreload
%autoreload 2

## Step 1: Link transform matrices 

Use matrices to take two basic shapes (a square and a wedge) and re-shape them into the geometry for the arm, gripper.

Yes, you could just create these basic shapes with the "correct" XYs, but we'll use a matrix to transform the
basic shape (square, wedge) to the correct size and shape.

This is actually what most packages (eg, solidworks) do when you make a model. Each part of the model is defined in a "canonical" location, then transformed to the desired position/scale/rotation using a matarix. This is
 *before* calculating the matrix that positions the part based on the, eg, joint angles

For all of these, you should be creating a matrix that consists of a scale followed by a rotate (maybe) followed by a translate

See slides for what the resulting re-positioned shapes look like

Wedge and Square are both -1,-1 to 1, 1 (you can open up the .json files to see what the vertex locations are)

TODO: Edit get_transform_base, get_transform_link, get_transform_palm, and (optional) get_transform_finger

In [ ]:
# Check the returned values
np.set_printoptions(precision=4, floatmode='fixed')  # Print out with 4 digits of precision

mat_base = get_transform_base(1.0, 0.5)
print(mat_base)
mat_base_check = np.array([[0, -0.5, 0], [0.25, 0.0, 0.25], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(mat_base, mat_base_check)))

In [ ]:
mat_link1 = get_transform_link(0.5, 0.25)
print(mat_link1)
mat_link1_check = np.array([[0.25, 0.0, 0.25], [0.0, 0.125, 0.0], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(mat_link1, mat_link1_check)))

In [ ]:
mat_palm = get_transform_palm(0.1)
print(mat_palm)
mat_palm_check = np.array([[0.005, 0.0, 0.0], [0.0, 0.05, 0.0], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(mat_palm, mat_palm_check)))

In [ ]:
mat_finger_top = get_transform_finger(0.1, (0.075, 0.025), True)
print(mat_finger_top)
mat_finger_top_check = np.array([[0.0375, 0.0, 0.0375], [0.0, 0.0125, 0.05], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(mat_finger_top, mat_finger_top_check)))

In [ ]:
mat_finger_bot = get_transform_finger(0.1, (0.075, 0.025), False)
mat_finger_bot_check = np.array([[0.0375, 0.0, 0.0375], [0.0, 0.0125, -0.05], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(mat_finger_bot, mat_finger_bot_check)))
print(mat_finger_bot)

In [ ]:
grader.check("Matrix_position")

# Put it all together to make the entire gripper

This should just "work" if you did the previous part correctly (no TODOs). 

In [ ]:
# Call it with example angles
base_size_param = (1.0, 0.5)
link_sizes_param = [(0.5, 0.25), (0.3, 0.1), (0.2, 0.05)]
palm_width_param = 0.1
finger_size_param = (0.075, 0.025)

# This function calls each of the set_transform_xxx functions, and puts the results
# in a list (the gripper - the last element - is a list)
arm_geometry = create_arm_geometry(base_size_param, link_sizes_param, palm_width_param, finger_size_param)
if len(arm_geometry) != 5:
    print("Wrong number of components, should be 5, got {len(arm_geometry)}")
if len(arm_geometry[-1]) != 3:
    print("Wrong number of gripper components, should be 3, got {len(arm_geometry[-1])}")

print(arm_geometry)

In [ ]:
grader.check("Step_1")

## Step 2 - set the matrices that rotate the links/palm

TODO: Edit get_rotation_link and (optional) get_matrix_finger

In [ ]:
# This part should be correct (it's from part 1)
base_size_param = (1.0, 0.5)
link_sizes_param = [(0.5, 0.25), (0.3, 0.1), (0.2, 0.05)]
palm_width_param = 0.1
finger_size_param = (0.075, 0.025)

angles_check = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [np.pi/3.0, np.pi/4.0, -np.pi/4.0]]

np.set_printoptions(precision=4, floatmode='fixed')

arm_geometry = create_arm_geometry(base_size_param, link_sizes_param, palm_width_param, finger_size_param)
set_angles_of_arm_geometry(arm_geometry, angles_check)

In [ ]:
# Check the rotation matrix for the first link
mat_rot_link1 = get_rotation_link(arm_geometry[1])
print(mat_rot_link1)
mat_rot_link1_check = np.array([[0.0, -1.0, 0.0], [1.0, 0.0, 0.0], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(mat_rot_link1, mat_rot_link1_check)))

In [ ]:
# Check the rotation matrix for the top finger (optional)
mat_rot_top_finger = get_matrix_finger(arm_geometry[-1][1])
print(mat_rot_top_finger)
mat_rot_top_finger_check = np.array([[0.7071, -0.7071, 0.0354], [0.7071, 0.7071, 0.0146], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(mat_rot_top_finger, mat_rot_top_finger_check, atol=0.3)))

In [ ]:
# Check the rotation matrix for the bottom finger (optional)
mat_rot_bot_finger = get_matrix_finger(arm_geometry[-1][2])
print(mat_rot_bot_finger)
mat_rot_bot_finger_check = np.array([[0.7071, 0.7071, 0.0354], [-0.7071, 0.7071, -0.0146], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(mat_rot_bot_finger, mat_rot_bot_finger_check, atol=0.3)))

In [ ]:
grader.check("set_matrix_from_angles")

<!-- BEGIN QUESTION -->

# Plot the arm (step 1)

Note: This should work for step 1 with **b_with_angles** set to False. You'll need to fix the TODOS in plot_arm_components to get it to draw properly with anglesfor step 2, in addition to doing the two functions in the previous problem

In [ ]:
# Re-defining here, just to be safe
base_size_param = (1.0, 0.5)
link_sizes_param = [(0.5, 0.25), (0.3, 0.1), (0.2, 0.05)]
palm_width_param = 0.1
finger_size_param = (0.075, 0.025)

# This function calls each of the set_transform_xxx functions, and puts the results
# in a list (the gripper - the last element - is a list)
arm_geometry = create_arm_geometry(base_size_param, link_sizes_param, palm_width_param, finger_size_param)

# Should show all 5 components, the base, 3 links, and the gripper
# Step 1 - note, comment out this one if you don't want both drawn on top of each other when you do step 2
fig, axs = plt.subplots(1, len(arm_geometry), figsize=(4 * len(arm_geometry), 4))

# TODO Edit plot_arm_components to do matrix transforms
plot_arm_components(axs, arm_geometry)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

# Plotting for step 2

I've included several different sets of angle values for you.

In [ ]:
# Repeating here
base_size_param = (1.0, 0.5)
link_sizes_param = [(0.5, 0.25), (0.3, 0.1), (0.2, 0.05)]
palm_width_param = 0.1
finger_size_param = (0.075, 0.025)

# This function calls each of the set_transform_xxx functions, and puts the results
# in a list (the gripper - the last element - is a list)
arm_geometry = create_arm_geometry(base_size_param, link_sizes_param, palm_width_param, finger_size_param)


In [ ]:
# Step 2 - rotate each link element in its own cooridinate system
# Several different angles to check your results with
angles_none = [0.0, 0.0, 0.0, [0.0, 0.0, 0.0]]
angles_check_fingers = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [0.0, np.pi/4.0, -np.pi/4.0]]
angles_check_wrist = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [np.pi/3.0, 0.0, 0.0]]
angles_check = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [np.pi/3.0, np.pi/4.0, -np.pi/4.0]]

# You can use the above set of angles to check each part of the plot
set_angles_of_arm_geometry(arm_geometry, angles_check)
fig2, axs2 = plt.subplots(1, len(arm_geometry), figsize=(4 * len(arm_geometry), 4))
plot_arm_components(axs2, arm_geometry, b_with_angles=True)

<!-- END QUESTION -->

## Hours and collaborators
Required for every assignment - fill out before you hand-in.

Listing names and websites helps you to document who you worked with and what internet help you received in the case of any plagiarism issues. You should list names of anyone (in class or not) who has substantially helped you with an assignment - or anyone you have *helped*. You do not need to list TAs.

Listing hours helps us track if the assignments are too long.

In [ ]:

# List of names (creates a set)
worked_with_names = {"not filled out"}
# List of URLS (creates a set)
websites = {"not filled out"}
# Approximate number of hours, including lab/in-class time
hours = -1.5

# for all row, column in all_indices_from_where
#.   if this is the column for wrist torque 
#.      print(f"Row: {r}, Time step: {c // n_time_steps} Successful y/n: {pick_data[r, -1] == 1}, value: {pick_data[r, c]}")

In [ ]:
grader.check("hours_collaborators")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Submit through gradescope, week 5 lab. Don't forget arm_routines.py

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)